<a href="https://colab.research.google.com/github/abbie-pillai/GenerativeAi/blob/main/Deepseek_Notebook_UNLV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro and Information
## Running Deepseep (DeepSeek-R1-Distill-Llama-8B) in Colab for free

Welcome to this guide on running Deepseep (DeepSeek-R1-Distill-Llama-8B) in Google Colab for free. In the following steps, you will learn how to configure your Colab runtime to use a GPU—an essential requirement for effectively running large language models. If you have any questions or need further assistance, please feel free to reach out at  [ryoung@unlv.edu](mailto:ryoung@unlv.edu)




### Don't Forget
Runtime → Change runtime type → Hardware accelerator: GPU.


logo.svg

[Deepseek Distilled Qwen 1.5B ](https://https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B)

## Install dependencies and show GPU info

---

In [1]:
!nvidia-smi

!pip install --quiet torch transformers accelerate gradio

Sun Jan 26 22:24:52 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Download from Huggingface

## [UNLV Huggingface](https://huggingface.co/UNLV)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Some Qwen-based models require trust_remote_code=True to properly load custom code.
# If you see "OSError: QWenTokenizer is not a valid ...", set trust_remote_code=True.
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Load the model onto the GPU in half precision
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # half-precision
    device_map="auto",           # automatically place layers on GPU
    trust_remote_code=True
)

# Optional: Ensure we're in eval mode (no training)
model.eval()

print("Model loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully!


# Using Gradio - a Web Ui

## How to use it
Simply click the generated link in the Colab output, and you’ll be taken to a web interface where you can input questions and see results. This interface is hosted temporarily while your Colab session is active.

## Remember to shut down your Colab session
Once you’ve finished using Gradio, ***be sure to shut down or turn off your Colab session***. This helps free up resources and ensures you don’t incur unnecessary usage of Colab’s free tiers.


In [3]:
import gradio as gr

def qwen_generate(prompt, max_new_tokens=128, temperature=0.7):
    """Generate text from the Qwen 1.5B model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=int(max_new_tokens),
            temperature=temperature,
            do_sample=True,
            top_p=0.95
        )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

demo = gr.Interface(
    fn=qwen_generate,
    inputs=[
        gr.Textbox(
            label="Prompt",
            lines=4,
            value="What is the difference between RNN and CNN?"
        ),
        gr.Slider(
            minimum=10,
            maximum=1024,
            value=128,
            step=8,
            label="Max New Tokens"
        ),
        gr.Slider(
            minimum=0.0,
            maximum=1.0,
            value=0.6,
            step=0.1,
            label="Temperature"
        ),
    ],
    outputs=gr.Textbox(label="Model Output", lines=10, max_lines=45,),
    title="DeepSeek-Qwen-1.5B Demo",
    description=(
        "Enter a prompt, adjust generation parameters, and click "
        "'Submit' to see the model's response.\n\n"
        "Presented by **Richard Young** (ryoung@unlv.edu)"
    ),

    examples=[
        ["Explain quantum computing in simple terms."],
        ["Write a short poem about artificial intelligence."],
        ["How do I train a neural network?"]
    ],

)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://703923fb17891a56da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
